<a href="https://colab.research.google.com/github/zzhining/financial_data_analysis/blob/main/finance_news_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요
- 네이버 증권 `뉴스` 탭의 3일치 기사를 수집하여 데이터 프레임으로 생성하고 파일로 저장
- 사이트 주소: https://finance.naver.com/news/mainnews.naver?date=2024-10-25

# 신문기사 목록 불러오기

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/news/mainnews.naver?date=2024-10-25'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

<!--  global include --><html lang="ko">
<head>
<title>네이버페이 증권</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="네이버페이 증권" name="apple-mobile-web-app-title"/>
<meta content="http://finance.naver.com/news/mainnews.naver" property="og:url"/>
<meta content="주요뉴스 : 네이버페이 증권" property="og:title"/>
<meta content="관심종목의 실시간 주가를 가장 빠르게 확인하는 곳" property="og:description"/>
<meta content="https://ssl.pstatic.net/static/m/stock/im/2016/08/og_stock-200.png" property="og:image"/>
<meta content="article" property="og:type"/>
<meta content="" property="og:article:thumbnailUrl"/>
<meta content="네이버페이 증권" property="og:article:author"/>
<meta content="http://FINANCE.NAVER.COM" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20241024180731/css/finance_header.css" rel="stylesheet" type="text/css"/>


- 데이터 프레임 구조
  - **기사제목**:  `<dd class="articleSubject">`안에 텍스트
  - **본문 URL**: `<dd class="articleSubject">` 안에 `<a>` 태그
  - **기사 본문**: `<div id="newsct_article" class="newsct_article _article_body">`
  - **날짜**: URL에서 가져오기

In [2]:
# 기사제목 리스트 가져오기
titles = soup.find_all('dd', {'class':'articleSubject'})
titles[0]

<dd class="articleSubject">
<a href="/news/news_read.naver?article_id=0005385669&amp;office_id=009&amp;mode=mainnews&amp;type=&amp;date=2024-10-25&amp;page=1">“찬바람 불길래 믿고 사놨는데”…오를 놈은 따로 있다는 보험주</a>
</dd>

In [4]:
# 첫번째 기사의 기사 제목
titles[0].text.strip()

'“찬바람 불길래 믿고 사놨는데”…오를 놈은 따로 있다는 보험주'

# 상세 뉴스 본문 읽어오기

## 상세 뉴스 URL 읽어오기

In [6]:
# 첫번째 기사의 상세페이지 URL
titles[0].find('a').get('href')

'/news/news_read.naver?article_id=0005385669&office_id=009&mode=mainnews&type=&date=2024-10-25&page=1'

In [7]:
news_url = 'https://finance.naver.com' + titles[0].find('a').get('href')
news_url

'https://finance.naver.com/news/news_read.naver?article_id=0005385669&office_id=009&mode=mainnews&type=&date=2024-10-25&page=1'

In [10]:
# 상세 뉴스페이지
news_response = requests.get(news_url)

news_soup = BeautifulSoup(news_response.text, 'lxml')
news_soup

<html><head><script>top.location.href='https://n.news.naver.com/mnews/article/009/0005385669';</script>
</head></html>

redirect 주소를 반환하고 있으므로, redirecet 주소를 사용하여 다시 상세 뉴스 페이지에 접근

In [17]:
news_url2 = news_soup.find('script').text.split("'")[1]

## 상세 뉴스 페이지 읽어오기

In [18]:
# 상세 뉴스페이지 재도전
news_response2 = requests.get(news_url2)

news_soup = BeautifulSoup(news_response2.text, 'lxml')
news_soup

<!DOCTYPE html>
<html data-useragent="python-requests/2.32.3" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="“찬바람 불길래 믿고 사놨는데”…오를 놈은 따로 있다는 보험주" property="og:title"/>
<meta content="article" property="og:type"/>
<meta content="https://n.news.naver.com/mnews/article/009/0005385669" property="og:url"/>
<meta content="https://imgnews.pstatic.net/image/009/2024/10/25/0005385669_001_20241025202307777.png?type=w800" property="og:image"/>
<meta content="계약 해지떄 돌려줄 준비금 회계기준 바뀌며 더 필요해 소형 보험주 배당여력 타격 삼성생명 올해 50% 뛸 때 현대해상·한화생명 제자리 올해 보험사들 사이에서 대형주와 중소형주 간 양극화 현상이 나타나고 " property="og:description"/>
<meta content="매일경제 | 네이버" property="og:article:author"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="“찬바람 불길래 믿고 사놨는데”…오를 놈은 따로 있다는 보험주" name="twitter:title"/>
<meta content="네이버 뉴스" 

## 상세 뉴스 페이지 본문 파싱하기

In [19]:
# 뉴스 본문
# 기사 본문: <div id="newsct_article" class="newsct_article _article_body">
news_soup.find('div', {'class':'newsct_article _article_body'}).text.strip()

'계약 해지떄 돌려줄 준비금회계기준 바뀌며 더 필요해소형 보험주 배당여력 타격삼성생명 올해 50% 뛸 때현대해상·한화생명 제자리\n\n\n\n올해 보험사들 사이에서 대형주와 중소형주 간 양극화 현상이 나타나고 있다. 대형 보험주는 연초 대비 30% 넘게 올랐으나, 중소형주는 상승률이 10%를 밑돌거나 심지어 주가가 떨어진 경우도 있다. 금융투자업계에서는 최근 새로운 국제회계기준 IFRS17이 도입되고 금융당국의 규제 강화로 중소형 보험사의 배당 여력이 떨어졌기 때문이라는 분석을 내놓고 있다.25일 한국거래소 정보데이터시스템에 따르면 올해 1월 2일~10월 25일 종가 기준 대형 보험주의 등락률은 삼성생명 49.12%, 삼성화재 37.12%, DB손해보험 39.22%로 상당히 높은 수치를 기록했다. 그러나 중소형 보험주는 현대해상 -0.82%, 한화생명 4.90%, 미래에셋생명 8.96% 등으로 저조한 수익률을 보였다.이에 국내 증시에서 거래되는 보험주 사이에는 시가총액이 높은 대형주만 더욱 커지고, 상대적으로 규모가 작은 소형주는 작아지는 양극화 현상이 나타나고 있다. 가령 시가총액이 높은 삼성생명(20조3400억원), 삼성화재(16조8891억원), DB손해보험(8조429억원) 등 대형주는 올해 상승률도 높았던 반면, 중소형주인 현대해상(2조7133억원), 한화생명(2조5101억원), 미래에셋생명(9258억원)은 상승률도 저조했다.이런 양극화의 원인으로는 금융당국이 2022년 새로 도입한 해약환급금준비금 제도에 따른 배당여력 양극화가 꼽힌다. 해약환급금 준비금은 고객들이 보험계약을 해약할 때 돌려줘야 하는 금액을 미리 쌓아놓는 것인데, 이는 배당 재원으로 활용할 수 없다.특히 지난해부터 IFRS17 회계 기준 도입으로 보험사의 부채를 원가가 아닌 시가 기준으로 평가하면서 해약환급금 준비금이 전반적으로 더 커졌다. 미래의 금리 변동이나 현금흐름의 불확실성 등을 반영해 더욱 보수적으로 해약환급금을 준비해야 하기 때문이다.정준섭 NH투자증권 연구원은 “양호한 당기순

# 반복문 사용하여 함수로 생성하기

In [35]:
import datetime
import time

In [36]:
def get_news_items(html):
  # 기사제목 리스트 가져오기
  titles = html.find_all('dd', {'class':'articleSubject'})
  title_list = []
  url_list = []
  article_list = []

  for t in titles:
    # 기사의 제목
    title = t.text.strip()

    # 기사의 상세페이지 URL
    news_url = 'https://finance.naver.com' + t.find('a').get('href')

    # 상세페이지 request
    news_response = requests.get(news_url)
    news_soup = BeautifulSoup(news_response.text, 'lxml')

    # 리다이렉트 주소를 파싱
    news_url2 = news_soup.find('script').text.split("'")[1]

    # 상세 뉴스페이지 request 재도전
    news_response2 = requests.get(news_url2)
    news_soup = BeautifulSoup(news_response2.text, 'lxml')

    # 신문기사 본문 파싱
    article = news_soup.find('div', {'class':'newsct_article _article_body'}).text.strip()

    # 리스트에 값 채우기
    title_list.append(title)
    url_list.append(news_url2)
    article_list.append(article)

  df = pd.DataFrame({'기사제목': title_list, '본문url': url_list, '기사본문': article_list})
  return df

In [37]:
news_df = pd.DataFrame()

for i in range(3):
    # 오늘 날짜 기준 i일 이전 날짜 구하기
    date= datetime.date.today() - datetime.timedelta(days = i)
    url = f'https://finance.naver.com/news/mainnews.naver?date={date}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    time.sleep(1)

    # 해당일 신문기사 스크랩 데이터프레임 반환
    temp_df = get_news_items(soup)
    temp_df['date'] = date
    news_df = pd.concat([news_df, temp_df], ignore_index=True).copy()

In [38]:
news_df.head()

,기사제목,본문url,기사본문,date
0,고려아연 “이번엔 장내매수 경쟁” 장중 29%대 폭등,https://n.news.naver.com/mnews/article/011/000...,전날 상한가에 이어 10.11% 상승영풍정밀 '계약해지' 소식에 -12%공개 매수 ...,2024-10-25
1,"[단독]현대차, '獨 디젤차 벌금' 870억 물어",https://n.news.naver.com/mnews/article/003/001...,2022년 현지 압수수색 결과배출가스 과다 배출…고의성은 없어 '과실' 판단\n\n...,2024-10-25
2,“찬바람 불길래 믿고 사놨는데”…오를 놈은 따로 있다는 보험주,https://n.news.naver.com/mnews/article/009/000...,계약 해지떄 돌려줄 준비금회계기준 바뀌며 더 필요해소형 보험주 배당여력 타격삼성생명...,2024-10-25
3,AI 열풍에도 주가 부진한 국내 IT사…'수혜주가 아니라 피해주',https://n.news.naver.com/mnews/article/015/000...,네이버·카카오를 비롯한 국내 인공지능(AI) 관련주 주가가 올초부터 장기 하락세를 ...,2024-10-25
4,고 이건희 4주기 '삼성 위기론' 속 조용한 추모…이재용 쇄신 카드는?,https://n.news.naver.com/mnews/article/057/000...,【 앵커멘트 】 고 이건희 삼성 선대회장의 4주기를 맞아 이재용 회장 등 삼성 일...,2024-10-25


In [39]:
news_df.tail()

,기사제목,본문url,기사본문,date
55,"""찬바람 부는 증시…고배당 우선주로 피신""",https://n.news.naver.com/mnews/article/015/000...,"변동성 장세 '투자 전략'우선주, 보통주보다 수익률 높아삼전 3.1% 하락때 삼전우...",2024-10-23
56,TI 매출 3분기째 줄었다 '아날로그 칩' 봄은 언제,https://n.news.naver.com/mnews/article/009/000...,"월가 ""수요위축 끔찍할 정도""TI·마이크로칩 목표가 내려\n\n\n\n세계 최대 아...",2024-10-23
57,트럼프든 해리스든 … 美국채 배신 계속된다,https://n.news.naver.com/mnews/article/009/000...,둘중 누가 되든 재정적자 악화월가 큰손들 잇단 공매도 선언10년물 금리 7월이후 최...,2024-10-23
58,부실채권 큰장 선다 … 전업사 3조 실탄 확보,https://n.news.naver.com/mnews/article/009/000...,상반기 은행서 3.5조 매각4분기 매각 더 늘어날 전망내년까지 NPL 시장 커질듯\...,2024-10-23
59,상한가 쳤다 원상 복귀…美대선 테마주 주의보,https://n.news.naver.com/mnews/article/015/000...,"비엘팜텍, 장 마감 직전 폭락여론조사 따라 급등락 반복국내 증시의 미국 대통령 선거...",2024-10-23


In [40]:
news_df.to_csv('article.csv')